## Quick setup

Recommended: use the project's environment managed by `uv`. Example commands:

```bash
uv sync
uv run python -m pip install plotly duckdb
```

If you prefer SQL-style analytics without importing CSVs into a database, `duckdb` is a great option (it can query CSVs directly).

In [ ]:
# Imports and versions
import sys, platform, subprocess
import pandas as pd
import plotly
import plotly.express as px
import duckdb

print('Python:', sys.version.splitlines()[0])
print('Pandas:', pd.__version__)
print('Plotly:', plotly.__version__)
print('DuckDB:', duckdb.__version__)

In [ ]:
# Paths to key CSVs
DATA_DOCS = 'document_stats.csv'
DATA_FILES = 'document_stats-files.csv'
DATA_LANG = 'Documents/language-check-report.csv'

# Load (these are small enough to load into memory in this repo)
docs = pd.read_csv(DATA_DOCS)
files = pd.read_csv(DATA_FILES)
lang = pd.read_csv(DATA_LANG)

docs.shape, files.shape, lang.shape

In [ ]:
# Quick heads and basic stats
display(docs.head())
display(docs.describe(include='all'))

display(files.head())
display(files['Pages'].describe())

display(lang.head())
print('Language-check issues by type:')
print(lang['Type'].value_counts().head(10))

In [ ]:
# Example plot: PDFs per subject (from document_stats.csv)
fig = px.bar(docs, x='Subject', y='PDFs', title='PDFs per subject')
fig.update_layout(xaxis={'categoryorder':'total descending'}, height=500)
fig.show()

In [ ]:
# Distribution of pages per file (document_stats-files.csv)
fig = px.histogram(files, x='Pages', nbins=30, title='Distribution of pages per file')
fig.show()

In [ ]:
# Join example: verify per-subject page totals
files_by_subject = files.groupby('Subject', as_index=False)['Pages'].sum().rename(columns={'Pages':'Pages_files'})
merged = pd.merge(docs, files_by_subject, on='Subject', how='left')
merged['Pages_diff'] = merged['Pages'] - merged['Pages_files']
display(merged[['Subject','Pages','Pages_files','Pages_diff']].sort_values('Pages_diff', key=abs, ascending=False).head(15))

In [ ]:
# Top subjects by language-check issues (quick summary)
top_issues = lang.groupby('Subject').size().reset_index(name='n').sort_values('n', ascending=False).head(25)
fig = px.bar(top_issues, x='Subject', y='n', title='Top subjects by language-check issues')
fig.update_layout(xaxis={'categoryorder':'total descending'}, height=500)
fig.show()

In [ ]:
# Save a small processed summary for quick reference
import os
os.makedirs('notebooks', exist_ok=True)
merged.to_csv('notebooks/processed_summary.csv', index=False)
print('Wrote notebooks/processed_summary.csv')

In [ ]:
# Reproducibility: git commit and environment
try:
    sha = subprocess.check_output(['git','rev-parse','--short','HEAD']).decode().strip()
except Exception:
    sha = '<not available>'
print('Git commit:', sha)
print('Platform:', platform.platform())

## Next steps

- Add a DuckDB-powered cell for SQL analytics without fully loading very large CSVs.
- Add more focused visualisations (issue types over time, heatmaps by rule ID, per-file issue density).
- Optionally add `nbval` or `papermill` based CI to execute this notebook in CI and ensure it runs without errors.